In [56]:
import torch
from torch import nn
import torch_geometric as pyg
import numpy as np

from torch_geometric.nn import Sequential
from torch_geometric.nn import GCNConv
from SimCKGConv import CKGConv
from tqdm import tqdm, trange
from torch_sparse import SparseTensor


## Prepare Synthetic Data

In [2]:
# Data
'''
        1 --- 0
       /       \
     /          \
    0            1
     \          /
      \        /
       1 --- 0
'''

adj = [
    [0, 1, 1, 0, 0, 0],
    [1, 0, 0, 1, 0, 0],
    [1, 0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 1],
    [0, 0, 1, 0, 0, 1],
    [0, 0, 0, 1, 1, 0]
]

x = y = torch.Tensor([1, 0, 0, 1, 1, 0]).view(-1, 1)
# y = torch.LongTensor([1, 0, 0, 1, 1, 0]).view(-1, 1)

adj = torch.Tensor(adj)

edge_index, edge_attr = pyg.utils.dense_to_sparse(adj)


In [47]:
deg = adj.sum(dim=1, keepdim=True)
rw = adj / deg.view(-1, 1)
out = torch.eye(adj.size(0))
rrwp = [out]
for i in range(5):
    out = out @ rw
    rrwp.append(out)
rrwp = torch.stack(rrwp, dim=-1)
rrwp_ = SparseTensor.from_dense(rrwp, has_value=True)
row, col, rrwp_val = rrwp_.coo()
rrwp_index = torch.stack([col, row], dim=0)

rrwp_val = rrwp_val * x.size(0)
pe_dim = rrwp_val.size(1)



In [33]:
rrwp_val

tensor([[6., 0.],
        [0., 3.],
        [0., 3.],
        [0., 3.],
        [6., 0.],
        [0., 3.],
        [0., 3.],
        [6., 0.],
        [0., 3.],
        [0., 3.],
        [6., 0.],
        [0., 3.],
        [0., 3.],
        [6., 0.],
        [0., 3.],
        [0., 3.],
        [0., 3.],
        [6., 0.]])

## Configuration

In [48]:
in_dim = 1
out_dim = 1
hid_dim = 16
max_ep = 200

device = 4


## 2-layer GCN

In [57]:
loss_ls = []
acc_ls = []

for r in range(5):
    num_layers = 2
    GCN = [(nn.Linear(in_dim, hid_dim), 'x -> x')]
    for i in range(num_layers):
        GCN += [(GCNConv(hid_dim, hid_dim, add_self_loops=True, normalize=True), 'x, edge_index -> x'),
                # (nn.BatchNorm1d(hid_dim), 'x -> x'),
                (nn.GELU(), 'x -> x')
                ]
    GCN += [(nn.Linear(hid_dim, out_dim), 'x -> x'),
            (nn.Sigmoid(), 'x -> x')]

    GCN = Sequential('x, edge_index', GCN)


    # ------- To Device -------
    GCN.to(device)
    opt = torch.optim.Adam(GCN.parameters(), lr=1e-3)
    loss_fn = nn.BCELoss()

    x, y = x.to(device), y.to(device)

    edge_index = edge_index.to(device)

    ### ----- Traing -------
    t_bar = trange(max_ep, desc='GCN', leave=True)
    for ep in t_bar:
        opt.zero_grad()
        pred = GCN.forward(x, edge_index)
        loss = loss_fn(pred, y)
        loss.backward()
        opt.step()
        pred_ = (pred >= 0.5).type(torch.long)
        acc = torch.mean((pred_ == y).type(torch.float))
        t_bar.set_description(f'GCN: bce_loss={loss.item()}  acc={acc.item()}')

    loss_ls.append(loss.item())
    acc_ls.append(acc.item())

print(f'loss:  {np.mean(loss_ls)}$\pm$ {np.std(loss_ls)}', )
print(f'acc:  {np.mean(acc_ls)}$\pm$ {np.std(acc_ls)}', )



GCN: bce_loss=0.14497503638267517  acc=1.0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 93.48it/s]

loss:  0.07138762921094895$\pm$ 0.04429862924070049
loss:  1.0$\pm$ 0.0


## 6-layer GCN

In [58]:

loss_ls = []
acc_ls = []

for r in range(5):
    num_layers = 6
    GCN = [(nn.Linear(in_dim, hid_dim), 'x -> x')]
    for i in range(num_layers):
        GCN += [(GCNConv(hid_dim, hid_dim, add_self_loops=True, normalize=True), 'x, edge_index -> x'),
                # (nn.BatchNorm1d(hid_dim), 'x -> x'),
                (nn.GELU(), 'x -> x')
                ]
    GCN += [(nn.Linear(hid_dim, out_dim), 'x -> x'),
            (nn.Sigmoid(), 'x -> x')]

    GCN = Sequential('x, edge_index', GCN)


    # ------- To Device -------
    GCN.to(device)
    opt = torch.optim.Adam(GCN.parameters(), lr=1e-3)
    loss_fn = nn.BCELoss()

    x, y = x.to(device), y.to(device)

    edge_index = edge_index.to(device)

    ### ----- Traing -------
    t_bar = trange(max_ep, desc='GCN', leave=True)
    for ep in t_bar:
        opt.zero_grad()
        pred = GCN.forward(x, edge_index)
        loss = loss_fn(pred, y)
        loss.backward()
        opt.step()
        pred_ = (pred >= 0.5).type(torch.long)
        acc = torch.mean((pred_ == y).type(torch.float))
        t_bar.set_description(f'GCN: bce_loss={loss.item()}  acc={acc.item()}')


    loss_ls.append(loss.item())
    acc_ls.append(acc.item())

print(f'loss:  {np.mean(loss_ls)}$\pm$ {np.std(loss_ls)}', )
print(f'acc:  {np.mean(acc_ls)}$\pm$ {np.std(acc_ls)}', )


GCN: bce_loss=0.693147599697113  acc=0.5: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:04<00:00, 40.76it/s]

loss:  0.693115520477295$\pm$ 2.225345299707741e-05
acc:  0.5$\pm$ 0.0


## 2-layer CKGCN

In [59]:

loss_ls = []
acc_ls = []

for r in range(5):
    num_layers = 2
    CKGCN = [(nn.Linear(in_dim, hid_dim), 'x -> x')]
    for i in range(num_layers):
        CKGCN += [(CKGConv(hid_dim, hid_dim, pe_dim, ffn_ratio=1., num_blocks=1, attn_dropout=0.), 'x, pe_index, pe_val -> x'),
                  # (nn.BatchNorm1d(hid_dim), 'x -> x'),
                  (nn.GELU(), 'x -> x')
                  ]
    CKGCN += [(nn.Linear(hid_dim, out_dim), 'x -> x'),
              (nn.Sigmoid(), 'x -> x')]

    CKGCN = Sequential('x, pe_index, pe_val', CKGCN)

    # ------- To Device -------
    CKGCN.to(device)
    opt = torch.optim.Adam(CKGCN.parameters(), lr=1e-3)
    loss_fn = nn.BCELoss()

    x, y = x.to(device), y.to(device)

    rrwp_index, rrwp_val = rrwp_index.to(device), rrwp_val.to(device)

    ### ----- Traing -------
    t_bar = trange(max_ep, desc='CKGCN', leave=True)
    for ep in t_bar:
        opt.zero_grad()
        pred = CKGCN.forward(x, rrwp_index, rrwp_val)
        loss = loss_fn(pred, y)
        loss.backward()
        opt.step()
        pred_ = (pred >= 0.5).type(torch.long)
        acc = torch.mean((pred_ == y).type(torch.float))
        t_bar.set_description(f'CKGCN: bce_loss={loss.item()}  acc={acc.item()}')


    loss_ls.append(loss.item())
    acc_ls.append(acc.item())

print(f'loss:  {np.mean(loss_ls)}$\pm$ {np.std(loss_ls)}', )
print(f'acc:  {np.mean(acc_ls)}$\pm$ {np.std(acc_ls)}', )


CKGCN: bce_loss=3.180006387992762e-05  acc=1.0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 88.89it/s]

loss:  3.723053996509407e-05$\pm$ 1.7522083686963205e-05
acc:  1.0$\pm$ 0.0


## 6-layer CKGCN

In [60]:

loss_ls = []
acc_ls = []

for r in range(5):

    num_layers = 6
    CKGCN = [(nn.Linear(in_dim, hid_dim), 'x -> x')]
    for i in range(num_layers):
        CKGCN += [(CKGConv(hid_dim, hid_dim, pe_dim, ffn_ratio=1., num_blocks=1, attn_dropout=0.), 'x, pe_index, pe_val -> x'),
                  # (nn.BatchNorm1d(hid_dim), 'x -> x'),
                  (nn.GELU(), 'x -> x')
                  ]
    CKGCN += [(nn.Linear(hid_dim, out_dim), 'x -> x'),
              (nn.Sigmoid(), 'x -> x')]

    CKGCN = Sequential('x, pe_index, pe_val', CKGCN)

    # ------- To Device -------
    CKGCN.to(device)
    opt = torch.optim.Adam(CKGCN.parameters(), lr=1e-3)
    loss_fn = nn.BCELoss()

    x, y = x.to(device), y.to(device)

    rrwp_index, rrwp_val = rrwp_index.to(device), rrwp_val.to(device)

    ### ----- Traing -------
    t_bar = trange(max_ep, desc='CKGCN', leave=True)
    for ep in t_bar:
        opt.zero_grad()
        pred = CKGCN.forward(x, rrwp_index, rrwp_val)
        loss = loss_fn(pred, y)
        loss.backward()
        opt.step()
        pred_ = (pred >= 0.5).type(torch.long)
        acc = torch.mean((pred_ == y).type(torch.float))
        t_bar.set_description(f'CKGCN: bce_loss={loss.item()}  acc={acc.item()}')


    loss_ls.append(loss.item())
    acc_ls.append(acc.item())

print(f'loss:  {np.mean(loss_ls)}$\pm$ {np.std(loss_ls)}', )
print(f'acc:  {np.mean(acc_ls)}$\pm$ {np.std(acc_ls)}', )


CKGCN: bce_loss=0.0  acc=1.0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:05<00:00, 34.57it/s]

loss:  0.0$\pm$ 0.0
acc:  1.0$\pm$ 0.0


tensor([[0., 1., 1., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 1.],
        [0., 0., 0., 1., 1., 0.]])